In [1]:
import numpy as np

def diag_2d(Y_prim_list):

    N_cols = 0

    for Y_prim in Y_prim_list:
        N_cols += Y_prim.shape[1]

    Y_prims = np.zeros((N_cols,N_cols))+0j

    it = 0
    for Y_prim in Y_prim_list:
        N = Y_prim.shape[0] 
        Y_prims[it:(it+N),it:(it+N)] = Y_prim
        it += N


    return Y_prims

def trafo_yprim(S_n,U_1n,U_2n,Z_cc,connection='Dyg11'):
    '''
    Trafo primitive as developed in: (in the paper Ynd11)
    R. C. Dugan and S. Santoso, “An example of 3-phase transformer modeling for distribution system analysis,” 
    2003 IEEE PES Transm. Distrib. Conf. Expo. (IEEE Cat. No.03CH37495), vol. 3, pp. 1028–1032, 2003. 
    
    '''

    if connection=='Dyn11':
        z_a = Z_cc*1.0**2/S_n*3
        z_b = Z_cc*1.0**2/S_n*3
        z_c = Z_cc*1.0**2/S_n*3
        U_1 = U_1n
        U_2 = U_2n/np.sqrt(3)
        Z_B = np.array([[z_a, 0.0, 0.0],
                        [0.0, z_b, 0.0],
                        [0.0, 0.0, z_c],])                             
        N_a = np.array([[ 1/U_1,     0],
                         [-1/U_1,     0],
                         [     0, 1/U_2],
                         [     0,-1/U_2]])           
        N_row_a = np.hstack((N_a,np.zeros((4,4))))
        N_row_b = np.hstack((np.zeros((4,2)),N_a,np.zeros((4,2))))
        N_row_c = np.hstack((np.zeros((4,4)),N_a))
        
        N = np.vstack((N_row_a,N_row_b,N_row_c))

        B = np.array([[ 1, 0, 0],
                      [-1, 0, 0],
                      [ 0, 1, 0],
                      [ 0,-1, 0],
                      [ 0, 0, 1],
                      [ 0, 0,-1]])
    
        Y_1 = B @ np.linalg.inv(Z_B) @ B.T
        Y_w = N @ Y_1 @ N.T
        A_trafo = np.zeros((7,12))

        A_trafo[0,0] = 1.0
        A_trafo[0,9] = 1.0
        A_trafo[1,1] = 1.0
        A_trafo[1,4] = 1.0
        A_trafo[2,5] = 1.0
        A_trafo[2,8] = 1.0

        A_trafo[3,2] = 1.0
        A_trafo[4,6] = 1.0
        A_trafo[5,10] = 1.0
        
        A_trafo[6,3] = 1.0
        A_trafo[6,7] = 1.0
        A_trafo[6,11] = 1.0
        
        Y_prim = A_trafo @ Y_w @ A_trafo.T
        
    if connection=='Ynd11':
        z_a = Z_cc*1.0**2/S_n*3
        z_b = Z_cc*1.0**2/S_n*3
        z_c = Z_cc*1.0**2/S_n*3
        U_1 = U_1n/np.sqrt(3)
        U_2 = U_2n
        Z_B = np.array([[z_a, 0.0, 0.0],
                        [0.0, z_b, 0.0],
                        [0.0, 0.0, z_c],])   

        B = np.array([[ 1, 0, 0],
                      [-1, 0, 0],
                      [ 0, 1, 0],
                      [ 0,-1, 0],
                      [ 0, 0, 1],
                      [ 0, 0,-1]])
                          
        N_a = np.array([[ 1/U_1,     0],
                        [-1/U_1,     0],
                        [     0, 1/U_2],
                        [     0,-1/U_2]])           
        N_row_a = np.hstack((N_a,np.zeros((4,4))))
        N_row_b = np.hstack((np.zeros((4,2)),N_a,np.zeros((4,2))))
        N_row_c = np.hstack((np.zeros((4,4)),N_a))
        
        N = np.vstack((N_row_a,N_row_b,N_row_c))

        Y_1 = B @ np.linalg.inv(Z_B) @ B.T
        Y_w = N @ Y_1 @ N.T
        A_trafo = np.zeros((7,12))
        A_trafo[0,0] = 1.0
        A_trafo[1,4] = 1.0
        A_trafo[2,8] = 1.0
        
        A_trafo[3,1] = 1.0
        A_trafo[3,5] = 1.0
        A_trafo[3,9] = 1.0
        
        A_trafo[4,2] = 1.0
        A_trafo[4,11] = 1.0
        A_trafo[5,3] = 1.0
        A_trafo[5,6] = 1.0
        A_trafo[6,7] = 1.0
        A_trafo[6,10] = 1.0
        
        
        Y_prim = A_trafo @ Y_w @ A_trafo.T
    
    return Y_prim

In [36]:
Z_cc = (0.01+0.04j)
U_1n = 20e3 
U_2n = 400.0
S_n = 500.0e3
Y_trafo_prim = trafo_yprim(S_n,U_1n,U_2n,Z_cc,connection='Dyn11')



In [37]:
Y_load_prim = np.eye(3)*1./10.0
Y_load_prim[0,0] = 1./10.0
Y_dummy_prim = np.eye(3)*1.0e-12
Y_th_prim = np.eye(3)*1e12
#Y_load_prim[0,0] = 10.0
#Y_load_prim[1,1] = 10.0

In [60]:
Y_prim = diag_2d([Y_th_prim,Y_dummy_prim,Y_trafo_prim,Y_load_prim])


### Incidence Matrix

In [51]:
id_nodes = np.array(['th_a','th_b','th_c','a_1','b_1','c_1','a_2','b_2','c_2','n_2'])
id_v = np.array(['th_a','th_b','th_c'])
id_i = np.array(['a_1','b_1','c_1','a_2','b_2','c_2','n_2'])

In [52]:
A = np.array([
  [ 1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # th_a 0
  [ 0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # th_b 1
  [ 0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # th_c 2
  [-1,   0,   0,   1,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0], # a_1  3
  [ 0,  -1,   0,   0,   1,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0], # b_1  4
  [ 0,   0,  -1,   0,   0,   1,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0], # c_1  5
  [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   1,   0,   0], # a_2  7
  [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   1,   0], # b_2  8
  [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   1], # c_2  9
  [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,  -1,  -1,  -1], # n_2 10
])
#   th            dummy          trafo                              load 2  

A_v = A[0:3,:]
A_i = A[3:11,:]

Y_ii = A_i @ Y_prim @ A_i.T
Y_iv = A_i @ Y_prim @ A_v.T
Y_vv = A_v @ Y_prim @ A_v.T
Y_vi = A_v @ Y_prim @ A_i.T

inv_Y_ii =  np.linalg.inv(Y_ii+1.0e-6)

In [53]:
V_known = np.zeros((3,1))+0j
V_known[id_v=='th_a'] = U_2/np.sqrt(3.0)*np.exp(0.0j)
V_known[id_v=='th_b'] = U_2/np.sqrt(3.0)*np.exp(2.0/3*np.pi*1j)
V_known[id_v=='th_c'] = U_2/np.sqrt(3.0)*np.exp(4.0/3*np.pi*1j)


I_known = np.zeros((7,1))+0j
I = 0.0
phi =np.deg2rad(0.0)
I_known[id_i=='a_1'] = I*np.exp((0.0-phi)*1j)
I_known[id_i=='b_1'] = I*np.exp((2.0/3.0*np.pi-phi)*1j)
I_known[id_i=='c_1'] = I*np.exp((4.0/3.0*np.pi-phi)*1j)
I_known[id_i=='n_1'] = 0.0
#I_known =0*np.array([1.0*np.exp(0.0j),
#                    1.0*np.exp(2.0/3*np.pi*1j),
#                    1.0*np.exp(4.0/3*np.pi*1j)]).reshape((9,1))

In [54]:
V_unknown = inv_Y_ii @ ( I_known - Y_iv @ V_known)
I_unknown =Y_vv @ V_known + Y_vi @ V_unknown

In [55]:
V_known

array([[ 11547.00538379    +0.j],
       [ -5773.50269190+10000.j],
       [ -5773.50269190-10000.j]])

In [59]:
for item in id_i:
    V = V_unknown[id_i==item,0] - V_unknown[id_i=='n_2',0] 
    print(item,': V = {:2.4f}|{:2.1f}º V'.format(float(np.abs(V)), float(np.angle(V,deg=True))))


a_1 : V = 11547.0054|0.0º V
b_1 : V = 11547.0054|120.0º V
c_1 : V = 11547.0054|-120.0º V
a_2 : V = 230.8660|-30.1º V
b_2 : V = 230.8660|89.9º V
c_2 : V = 230.8660|-150.1º V
n_2 : V = 0.0000|0.0º V


In [57]:
for item in id_v:
    I = I_unknown[id_v==item,0]
    print(item,': I = {:2.4f}|{:2.1f}'.format(float(np.abs(I)), float(np.angle(I,deg=True))))

for item in id_i:
    I = (V_unknown[id_i==item,0] - V_unknown[id_i=='n_1',0])/10.0
    print(item,': I = {:2.4f}|{:2.1f}º V'.format(float(np.abs(I)), float(np.angle(I,deg=True))))

th_a : I = 0.0006|-90.0
th_b : I = 0.0000|0.0
th_c : I = 0.0000|0.0


TypeError: only length-1 arrays can be converted to Python scalars

In [35]:
np.abs(I_unknown.T @ A_v)

array([[    0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,  1271.88398431,  1273.43921331,
         2518.67230305,  2518.67230305,  3781.87554658,  3783.44500432]])

In [75]:
a_1 : V = 143.1010|-69.0º V
b_1 : V = 230.8814|89.9º V
c_1 : V = 230.8814|-150.1º V
n_1 : V = 0.0000|0.0º V
a_2 : V = 11509.8621|-0.0º V
b_2 : V = 11561.7395|120.2º V
c_2 : V = 11569.5053|-120.1º V
a_1 : V = 143.1010|-69.0º V
b_1 : V = 230.8814|89.9º V
c_1 : V = 230.8814|-150.1º V
n_1 : V = 0.0000|0.0º V
a_2 : V = 11509.8621|-0.0º V
b_2 : V = 11561.7395|120.2º V
c_2 : V = 11569.5053|-120.1º V
    
th_a : I = 165.0642|-68.9
th_b : I = 165.2755|111.3
th_c : I = 0.0000|0.0
a_1 : I = 14.3101|-69.0º V
b_1 : I = 23.0881|89.9º V
c_1 : I = 23.0881|-150.1º V
n_1 : I = 0.0000|0.0º V
a_2 : I = 1150.9862|-0.0º V
b_2 : I = 1156.1740|120.2º V
c_2 : I = 1156.9505|-120.1º V


SyntaxError: invalid syntax (<ipython-input-75-b7cf56bf7a45>, line 1)

In [76]:
20e3**2/630.0e3

634.9206349206349

In [52]:
0.04*635

25.400000000000002

In [ ]:
20000/1.73/25.4

In [ ]:
1/60


In [ ]:
0.92-0.88

In [19]:
import numba

In [29]:
@numba.jit(nopython=True)
def hola_strings(s_list):
    return s_list
    
    

In [30]:
s_list = np.array(['a_1','b_1','c_1','n_1','a_2','b_2','c_2','th_a','th_b','th_c'])

hola_strings('a_1')


NotImplementedError: Failed at nopython (nopython mode backend)
cannot convert str to native value

In [31]:
c=1200/3.6

In [33]:
c/200

1.6666666666666665

In [34]:
333/0.8


416.25

In [87]:
np.linalg.inv(Y_trafo_prim)

LinAlgError: Singular matrix